# Advent of Code 2022: Day 17
https://adventofcode.com/2022/day/17


## Part 1
Calculate the height of a tower after dropping 2022 blocks.


Get the data into a list of strings

In [3]:
myfile = open('input.txt', 'r')
data = myfile.read()
data_list = data.split('\n')
# Remove empty value at the bottom of the list.
data_list = data_list[:-1]

### Get the list of moves, and create a list of rock shapes

In [4]:
move_list = list(data_list[0])

In [5]:
rock_shapes = ['-', '+', 'J', 'l', 'D']

### Function to drop a block

In [6]:
def drop_block(board, block, move_list, heights, max_height, pat=0, find_pattern=False):
  
  # Get the desired height of the board
  # based on the current max height. 
  # Insert empty rows at the top of 
  # the board until the board is 
  # h+3 tall, to account for 
  # the tallest possible rock shape.
  h = max_height +4 
  while len(board) < h+3:
    board.insert(0, ['.','.','.','.','.','.','.'])
  
  # Setup a list of coordinates as the
  # initial position of a block depending
  # on the provided shape.
  if block == '-':
    pos = [[h, 2], [h,3], [h,4], [h,5]]
  elif block == '+':
    pos = [[h+1, 2], [h,3], [h+2, 3], [h+1, 4]]
  elif block == 'J':
    pos = [[h,2], [h,3], [h,4], [h+1, 4], [h+2, 4]]
  elif block == 'l':
    pos = [[h, 2], [h+1, 2], [h+2, 2], [h+3, 2]]
  elif block == 'D':
    pos = [[h,2], [h,3], [h+1,2], [h+1,3]]
  
  # Setup a counter for dropping blocks
  # and a counter for the pattern checking
  # if turned on.
  i = 0
  pat = pat
  
  # While a block can still move
  # execute moves if i is an odd
  # number and drop it down one
  # level if i is an even number.
  while True:
    i += 1
    if i % 2 != 0:
      
      # Add to the pattern
      # counter if turned on.
      if find_pattern==True:
        pat +=1
      
      # Get the current move and put 
      # it at the back of the move list.
      curr_move = move_list.pop(0)
      move_list.append(curr_move)

      # Based on which move to carry out,
      # initialize it to a valid move and
      # perform two checks for each position
      # of a block: if a move in this direction
      # would put it outside the board, or if
      # there is already something in the
      # position where it tries to move. 
      # If either of these cases, set valid
      # to false and break the loop. If every
      # position passed the check, carry out
      # the move.
      if curr_move == '<':
        valid = True
        for p in pos:
          if p[1]-1 < 0 or board[len(board)-p[0]][p[1]-1] != '.':
            valid = False
            break

        if valid == True:
          for p in range(len(pos)):
            pos[p][1] -=1

      elif curr_move == '>':
        valid = True
        for p in pos:
          if p[1]+1 > 6 or board[len(board)-p[0]][p[1]+1] != '.':
            valid = False
            break        

        if valid == True:
          for p in range(len(pos)):
            pos[p][1] +=1
    
    # For a downward move, there are likewise
    # two checks to perform for each position:
    # if a block is already at the bottom of
    # the board, or if there is already
    # something below the block.
    elif i % 2 == 0:
      valid = True
      for p in pos:
        if p[0]-1 <= 0:
          valid = False
          break
        elif board[len(board)-p[0]+1][p[1]] != '.':
          valid = False
          break

      # Similarly to the moves, if every check 
      # passed, the block is allowed to fall by
      # one level. However, if a the move is not 
      # valid, this means the block should stop.
      # If this is the case, we go through each
      # position of the block and update a list
      # that holds the maximum height for every
      # column. The board is also updated by
      # setting each position of the block to '#'.
      # Finally, the function returns the updated
      # heights list, board and pattern if turned on.   
      if valid == True:
        for p in range(len(pos)):
          pos[p][0] -=1
      elif valid == False:
        for p in pos:
          if p[0] > heights[p[1]]:
            heights[p[1]] = p[0]
          board[len(board)-p[0]][p[1]] = '#'
        
        if find_pattern==True:
          return heights, board, pat
        return heights, board      

### Function to count the height of a tower of blocks.

In [7]:
def count_height(n_rocks, find_pattern = False):
  
  # Initialize the rock shapes and move list
  # to ensure that the result is always the same.
  # Also setup the board and heights list.
  rock_shapes = ['-', '+', 'J', 'l', 'D']
  move_list = list(data_list[0])
  board = [['.' for i in range(7)]]
  heights = [0 for i in range(7)]
  
  # If we should not look for a pattern, then 
  # loop for the amount of rocks that should be 
  # dropped and return the maximum height.
  if find_pattern == False:
    for i in range(n_rocks):
      block = rock_shapes.pop(0)
      rock_shapes.append(block)
      heights, board = drop_block(board, block, move_list, heights, max(heights))
    return max(heights)
  
  # If the function should look for a pattern, 
  # then initialize the pattern list, current 
  # pattern, and two variables for storing
  # the previous values of i and the heights list.
  elif find_pattern == True:
    pattern_list = []
    pat = 0
    last_i = 0
    last_h = [0,0,0,0,0,0,0]
    
    # Start by going through the entire
    # move list and drop that number of blocks. 
    for i in range(len(move_list)):
      block = rock_shapes.pop(0)
      rock_shapes.append(block)
      heights, board, pat = drop_block(board, block, move_list, heights, max(heights), pat, find_pattern)

      # If the pattern counter is equal to or
      # greater than the length of the move list,
      # then get a copy of the current height list
      # and calculate the difference in height
      # between now and the last time a pattern was 
      # encountered. Append this list of differences
      # to the pattern list, along with the current
      # pattern counter and the difference in i.
      # Finally reset the pattern counter to 0, and
      # update the last_i and last_h.
      if pat-len(move_list) >= 0:
        h = heights.copy()
        h_diff = []
        for he in range(len(h)):
          h_diff.append(h[he] - last_h[he])
        pattern_list.append([h_diff, pat, i-last_i])
        pat = 0
        last_i = i
        last_h = h

    # With the pattern found, reset the list of
    # rock shapes, list of moves, board, and list
    # of heights. As the first iteration of the 
    # pattern differs from the rest, start by dropping
    # blocks for that number of blocks + the number of
    # blocks for the first of the regular patterns that
    # follow. The reason for this is that after the unique
    # pattern the list of rock shapes and moves have not
    # looped back around to their initial states. 
    rock_shapes = ['-', '+', 'J', 'l', 'D']
    move_list = list(data_list[0])
    board = [['.' for i in range(7)]]
    heights = [0 for i in range(7)]
    for i in range(pattern_list[0][2]+pattern_list[1][2]):
      block = rock_shapes.pop(0)
      rock_shapes.append(block)
      heights, board = drop_block(board, block, move_list, heights, max(heights))

    # Calculate the pattern count as the number of rocks
    # to be dropped, minus the number of blocks in the initial 
    # unique pattern, divided by the number of blocks in the regular
    # pattern. Subtract by 1 to account for already doing one regular
    # pattern.  
    pattern_count = ((n_rocks - pattern_list[0][2]) // pattern_list[1][2])-1
    
    # As the number of rocks is not divisible by the pattern length,
    # a number of blocks still have to be dropped after the final pattern.
    # Calculate this number and drop that amount of blocks.
    for i in range(n_rocks-(((pattern_count+1)*pattern_list[1][2])+pattern_list[0][2])):
      block = rock_shapes.pop(0)
      rock_shapes.append(block)
      heights, board = drop_block(board, block, move_list, heights, max(heights))
    
    # Finally, take the heights and add on for each column add the
    # height for the the pattern count to simulate dropping these blocks.
    for h in range(len(heights)):
      heights[h] = heights[h] + (pattern_list[1][0][0]*pattern_count)
   
      
    return max(heights)

In [8]:
count_height(2022)

3163

## Part 2


In [9]:
count_height(1000000000000, True)

1560932944615